<a href="https://colab.research.google.com/github/investingthemovie/PublicWork/blob/main/NER_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.2 MB/s eta 0:00:00


## Load the transcript dataset

In [ ]:
import torch
import pandas as pd

In [ ]:
file_url = 'https://drive.google.com/uc?id=' + '1hiCOBxEUwe_b31n-fyh_lMHQgqnxrQqP'
df = pd.read_pickle(file_url)

In [ ]:
df.sample(10)

,videoId,url,title,view,transcript,Channel
1168,1TslhBmVqyA,https://www.youtube.com/watch?v=1TslhBmVqyA,Iphone 7 vs Canon 80D Side by Side Video Test!,2017-12-19T00:36:38Z,well welcoming guys today we're gonna be testi...,MeetKevin
30,kKLe-95VkMM,https://www.youtube.com/watch?v=kKLe-95VkMM,robinhood is suing me...,2020-04-02T00:25:52Z,well it looks like I ruffled some feathers I'm...,FinEd1
1175,a3-oMIEBLuU,https://www.youtube.com/watch?v=a3-oMIEBLuU,"Tesla Stock to $0 or $1,000? Why it matters to...",2019-07-26T14:39:42Z,oh we're live oh my goodness we are live we ar...,MeetKevin
268,O_uPG3Ex4qM,https://www.youtube.com/watch?v=O_uPG3Ex4qM,Top 3 Stocks NOW 🚀| June 2020,2020-06-08T01:36:10Z,after capping off another record week in the s...,FinEd1
159,yQaq_wuhDKU,https://www.youtube.com/watch?v=yQaq_wuhDKU,WHAT IS COMING [DETAILS....],2022-01-09T23:48:19Z,okay folks so we've got lots to talk about tod...,FinEd1
1570,Jq81MP-qpVI,https://www.youtube.com/watch?v=Jq81MP-qpVI,WHY STOCKS ARE ABOUT TO GO INSANE [details...],2021-04-07T01:35:03Z,okay so we're at a point where almost every ma...,FinEd2
587,Nt2aoZX8k-0,https://www.youtube.com/watch?v=Nt2aoZX8k-0,Warning: Pay Attention to This UNEXPECTED Shift.,2022-08-17T14:11:36Z,hey everyone me kevin here in january we talke...,FinEd1
509,IB4KDtb8wgs,https://www.youtube.com/watch?v=IB4KDtb8wgs,Watch what Joe Biden did during a press confer...,2022-06-04T07:00:07Z,you won't believe what joe biden just pulled o...,FinEd1
337,bYfsV54TJzg,https://www.youtube.com/watch?v=bYfsV54TJzg,Top 3 Stocks NOW 🚀 | May 2021,2021-05-10T02:06:05Z,after another week in the battle of the stock ...,FinEd1
90,nCdyShLaMYM,https://www.youtube.com/watch?v=nCdyShLaMYM,*WOW! WATCH FAST*,2022-11-23T01:29:36Z,folks the entire banking system right now is g...,FinEd1


In [ ]:
df.shape

(1724, 6)

## Load the S&P 500 company/ticker dataset

In [ ]:
file_url = 'https://drive.google.com/uc?id=' + '1JGg5CIGHUWXPUd3Dg39Nj7kqhRTtCbMM'
company_df = pd.read_csv(file_url)

In [ ]:
company_df.sample(10)

,Symbol,Name,Sector
363,PBCT,People's United Financial,Financials
462,UPS,United Parcel Service,Industrials
426,SYY,Sysco,Consumer Staples
26,AMZN,Amazon,Consumer Discretionary
77,AVGO,Broadcom,Information Technology
378,PLD,Prologis,Real Estate
224,HCA,HCA Healthcare,Health Care
437,TXN,Texas Instruments,Information Technology
345,NVR,NVR,Consumer Discretionary
55,AZO,AutoZone,Consumer Discretionary


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertForSequenceClassification, BertTokenizer, pipeline

ner_tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

sent_tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
sent_model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

ner_pipeline = pipeline('ner', model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple", device=0)
sent_pipeline = pipeline('sentiment-analysis', model=sent_model, tokenizer=sent_tokenizer, device=0, return_all_scores=True)

sent_tokenizer_kwargs = {'padding':True, 'truncation':True, 'max_length':512}

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
%%time

CHUNKSIZE = 512
companies = list(company_df['Name'].str.lower())
result = []

# Iterate through the dataframe
for row in range(50): # range(df.shape[0]):
        
    transcript = df.iloc[row,4].split(' ')
    ts = df.iloc[row,3]
    url = df.iloc[row,1]
    
    # Divide the transcripts into chunks of 512 words
    text = [transcript[i:i + CHUNKSIZE] for i in range(0, len(transcript), CHUNKSIZE)]
    # Iterate through the chunks
    for txt in text:
      # Signal used to track if we have calculated the sentiment yet
      sent_calculated = False
      txt = ' '.join(txt)
      # Pass the transcript to the NER pipeline
      ner_result = ner_pipeline(txt)
      # Only continue if there is NER result
      if ner_result:
        for entity in ner_result:
          if entity['entity_group'] == 'ORG':
              # Only continue if the entity is org and in our companies list
              if entity['word'].lower() in companies:
                # Calculate the sentiment if we haven't done it yet
                if not sent_calculated:
                  sent_result = sent_pipeline(txt, **sent_tokenizer_kwargs)[0]
                  for sent in sent_result:
                    if sent['label'] == 'positive':
                      pos_score = sent['score']
                    elif sent['label'] == 'negative':
                      neg_score = sent['score']
                  sent_calculated = True
                # Save everything is a Python dict and append it to the result list
                item = {'source_ts': ts, 'source_url': url, 'source_entity': entity['word'], 
                        'pos_score': pos_score, 'neg_score': neg_score}
                result.append(item)
    
    if row % 10 == 1:
      print(f'Finished {row} lines')

Finished 1 lines


/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Finished 11 lines
Finished 21 lines
Finished 31 lines
Finished 41 lines
CPU times: user 11.9 s, sys: 86.5 ms, total: 12 s
Wall time: 14 s


In [ ]:
sentiment_df = pd.DataFrame(result)

In [ ]:
sentiment_df.shape

(59, 5)

In [ ]:
sentiment_df.sample(10)

,source_ts,source_url,source_entity,pos_score,neg_score
31,2022-10-23T23:39:36Z,https://www.youtube.com/watch?v=K4MWW3IOHHA,Amazon,0.041191,0.164267
27,2022-10-23T23:39:36Z,https://www.youtube.com/watch?v=K4MWW3IOHHA,Amazon,0.041191,0.164267
5,2022-10-14T23:10:02Z,https://www.youtube.com/watch?v=0Z1DWagQwKM,Bank of America,0.018239,0.918903
38,2023-01-03T22:51:43Z,https://www.youtube.com/watch?v=hB8pUhMjQNk,Tesla,0.024586,0.637817
6,2022-09-15T01:33:03Z,https://www.youtube.com/watch?v=bg00TJvElNg,Goldman Sachs,0.041737,0.121611
52,2022-12-20T01:58:03Z,https://www.youtube.com/watch?v=RyB5NWzuABc,Morgan Stanley,0.015912,0.871149
25,2022-10-23T23:39:36Z,https://www.youtube.com/watch?v=K4MWW3IOHHA,Boeing,0.125282,0.051485
29,2022-10-23T23:39:36Z,https://www.youtube.com/watch?v=K4MWW3IOHHA,apple,0.041191,0.164267
42,2023-01-03T22:51:43Z,https://www.youtube.com/watch?v=hB8pUhMjQNk,Tesla,0.024586,0.637817
41,2023-01-03T22:51:43Z,https://www.youtube.com/watch?v=hB8pUhMjQNk,Tesla,0.024586,0.637817


In [14]:
sentiment_df.to_pickle('./sentiment_df.pkl')